# LeakGAN

Цели:

- Разобраться с основными архитектурными решениями и алгоритмом обучения LeakGAN
- Реализация LeakGAN для задачи посимвольной генерации текстов

Содержание:

* [Введение](#intro)
* [Загрузка данных](#load_data)
* [Алгоритм](#algorithm)
  * История возникновения иерархического обучения с подкреплением
  * Нейросети для иерархического RL
  * Методология LeakGAN
* [Реализация](#implementation)
  * [Загрузчики данных](#data_loaders)
  * [Генератор](#generator)
  * [Дискриминатор](#discriminator)
  * [Предобучение генератора (MLE)](#gen_pretrain)
  * [Предобучение дискриминатора](#disc_pretrain)
  * [Обучение в состязательном режиме](#adversarial_train)

Ссылки:

* [Long Text Generation via Adversarial Training with Leaked Information](https://browse.arxiv.org/pdf/1709.08624v2.pdf)
* [FeUdal Networks for Hierarchical Reinforcement Learning](https://arxiv.org/pdf/1703.01161.pdf)
* [Feudal Reinforcement Learning](https://proceedings.neurips.cc/paper/1992/file/d14220ee66aeec73c49038385428ec4c-Paper.pdf)
* [Adversarial Feature Matching for Text Generation (TextGAN)](https://arxiv.org/pdf/1706.03850.pdf)
* [Реализация, адаптированная в этом ноутбуке](https://github.com/nurpeiis/LeakGAN-PyTorch/blob/master)
* [Реализация в состaве framework с десятком разных GAN](https://github.com/williamSYSU/TextGAN-PyTorch)

<a name="intro"></a>
## Введение

### Previously on

Генерация текста моделируется как последовательный процесс принятия решений,
где на каждом шаге принимаются решения относительно следующего слова. В этом процессе состояние представляет уже сгенерированные слова, действие - выбор следующего слова, а генератор - случайная стратегия (stochastic policy), которая отображает состояние в распределение вероятностей по следующим действиям (следующим словам).

После завершения генерации текста, полученные тексты проходят через дискриминатор, это классификатор, который обучен отличать настоящий текст от сгенерированного. Это позволяет получить сигналы обратной связи для обновления параметров генератора.



### Предпосылки для LeakGAN

Существующие методы генерации текстов (до конца 2017) имеют недостаток в том, что управляющий сигнал от дискриминатора приходит достаточно редко и доступен только после генерации всей последовательности текста. К тому же, скалярный управляющий сигнал для всего текста может быть недостаточно информативным. Нам (генератору) в общем-то для успешного обучения необходимо понять внутреннюю синтаксическую структуру и семантику сгенерированного текста, но дают одно число в час по чайной ложке (по окончанию генерации предложения).

Для того, чтобы сделать управляющий сигнал более информативным, дискриминатор может предоставлять больше информации, чем просто финальное вознаграждение, так как сам дискриминатор также является моделью и не является "чёрным ящиком". Здесь отсылочка к тому, что в большинстве традиционных RL-сред мы не понимаем внутренней механизмов или динамики среды и как формируется вознаграждение - оно нам просто выдаётся по окончанию эпизода. Например в робототехнике, играх (Dota 2, Starcraft) или автономном вождении.

Именно на идее более информативного сигнала основан метод [TextGAN](https://arxiv.org/pdf/1706.03850.pdf). В TextGAN при обучении генератора скрытые представления настоящих и сгенерированных текстов вынуждаются быть похожими. Для генерации коротких текстов такой подход может быть эффективным, но управляющие сигналы по прежнему появляются только после генерации текста целиком.

С другой стороны для решения проблемы разреженности управляющего сигнала (редкого вознаграждения) возникает идея иерархии, когда мы декомпозируем сложную задачу генерации на более мелкие подзадачи и модели становится легче их выучить (своего рода разделяй-и-властвуй).


### Обобщённая архитектура LeakGAN

<img src="https://raw.githubusercontent.com/sswt/dive2gai/main/.github/images/w4/leakgan_arch.png" alt="SeqGAN" width="60%" />

LeakGAN должен устранить как слабую информативность вознаграждения, так и разреженность сигнала. Из идей иерархического RL вводится иерархический генератор, который состоит из высокоуровневого модуля MANAGER и низкоуровневого модуля WORKER.

MANAGER - LSTM, работает как посредник. На каждом шаге, он принимает признаковое представление из дискриминатора и использует для формирования цель-ориентир (guiding goal) для WORKER на текущем шаге. Это внутренняя информация дискриминатора и неявно содержит часть информации о следующем токене, поэтому это назвается ликом. На kaggle обычно борются с ликами, а здесь их используют во благо.

Затем, на основе эмбеддинга цели от менеджера, работник кодирует уже сгенерированные слова своей внутренней LSTM и совмещает выход с эмбеддингом цели, чтобы принять решение о действии в текущем состоянии.

Таким образом, управляющие сигналы от дискриминатора доступны генератору не только в конце, как скалярное вознаграждение, но и в качестве вектора эмбеддинга цели непосредственно в процессе генерации, чтобы генератору было легче понять, как улучшаться.

<a name="load_data"></a>
## Загрузка данных

In [ ]:
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import namedtuple

import numpy as np

In [ ]:
!wget -q https://huggingface.co/datasets/sswt/arxiv_sample_50K/resolve/main/title_summary_ascii.txt.tar.gz -O data.tar.gz && tar -xzf data.tar.gz

In [ ]:
max_len = 64
with open('title_summary_ascii.txt') as fp:
    lines = [line[:max_len] for line in fp.readlines()]

In [ ]:
lines[:3]

[' On Finitely Generated Models of Theories with at Most Countably',
 ' Generalized modeling of ecological population dynamics ; Over t',
 ' Generating Subsurface Earth Models using Discrete Representatio']

In [ ]:
vocab = {c: i for i, c in enumerate(sorted(set([c for l in lines for c in l])))}
inv_vocab = {i: c for c, i in vocab.items()}
vocab_size = len(vocab)
len(vocab)

68

In [ ]:
''.join(vocab)

"\n ',.0123456789;ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [ ]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
def to_tensor(line):
    t = torch.LongTensor([vocab[line[li]] for li in range(len(line[:max_len]))])
    if len(line) < max_len:
        t = nn.ConstantPad1d((0, max_len - len(line)), 0)(t)
    return t

def to_text(t):
    return ''.join(inv_vocab[i] for i in t.tolist())

In [ ]:
sequences = torch.stack([to_tensor(lines[i]) for i in range(6400)])
np.save('./data/train_corpus.npy', sequences.numpy())

<a name="algorithm"></a>
## Алгоритм



### История возникновения иерархического обучения с подкреплением

Первые успехи нейросетей на поприще RL были достаточно давно. В 1992 году Джон Тезауро разработал программу TD-Gammon, которая играла в нард. Так как в нардах ход игры зависит от броска кубиков и в стратегию игры всегда вносится элемент случайности, что во многом определяет стратегию, то алгоритму можно было тренировать модель просто играя самому с собой. Обобщить это на другие игры тогда не удалось и в качестве одной из идей в работе Feudal Reinforcement Learning (в соавторстве с Хинтоном) было озвучено иерархическое RL.

Выдержка из текста статьи:

> Мы стремились создать систему, которая отражает иерархические аспекты феодальной вассальной феодальной системы, как один из крайних случаев моделей управления. Менеджерам предоставляется абсолютная власть над своими подчиненными менеджерами - они могут назначать им задания и награждать или наказывать их так, как им угодно. Однако менеджерам в конечном итоге приходится удовлетворять своих собственных вышестоящих менеджеров, иначе они сами подвергаются наказанию - и таким образом, происходит рекурсивное укрепление и отбор, пока вся система не достигнет цели высшего уровня менеджера. Это можно добиться без того, чтобы подчиненные менеджеры сначала "понимали" подзадачи, которые им назначают. Каждый компонент просто действует, чтобы максимизировать ожидаемое вознаграждение, поэтому после обучения значение, которое он прикрепляет к описанию подзадачи, заключается в том, как это описание влияет на его выбор под-под-менеджеров и под-под-задач.

Два принципа играют ключевую роль:

* Скрытие вознаграждения
  
  Менеджеры должны награждать подчиненных менеджеров за выполнение своих заданий, независимо от того, удовлетворяют ли они команды вышестоящих менеджеров. Подчиненные менеджеры должны просто учиться подчиняться своим менеджерам и позволить им решить, что лучше всего делать на следующем уровне. Таким образом, если подчиненный менеджер не достигает подцели, установленной его менеджером, его не награждают, даже если его действия приводят к удовлетворению цели менеджера. В начальной стадии обучения менеджеры низкого уровня могут стать довольно компетентными в достижении целей низкого уровня, даже если цель высшего уровня никогда не была удовлетворена.

* Скрытие информации

  Менеджерам нужно знать состояние системы с точностью до их собственного выбора задач. Предоставление некоторых решений на более грубом уровне - одна из основных целей иерархической декомпозиции. Информация скрывается как вниз по иерархии - подчиненные менеджеры не знают о задаче, которую высший менеджер назначил менеджеру, так и вверх по иерархии - высший менеджер не знает, какие решения принял его менеджер для выполнения его команд. Однако менеджерам нужно знать условия удовлетворения для задач, которые они устанавливают, а также некоторую меру фактической стоимости для системы при их выполнении с использованием подчиненных менеджеров и задач, которые они выбрали в конкретный момент.
  Для особого случая, рассматриваемого здесь, когда менеджерам не предоставляется выбора, какого подчиненного менеджера использовать в данном состоянии, их выбор задачи очень похож на действие в стандартной системе Q-обучения. Если задача успешно выполнена, стоимость определяется высшим менеджером в зависимости от того, насколько хорошо (например, насколько быстро или вообще) менеджер удовлетворяет его супер-задачи. В зависимости от того, как выполняется их собственная задача, менеджер награждает или наказывает подчиненного менеджера. Когда менеджер выбирает действие, управление передается подчиненному менеджеру и возвращается только тогда, когда состояние меняется на уровне управления.

Феодальная архитектура частично решает одну из основных проблем в обучении с подкреплением - вопрос о том, как разбить одну задачу на подзадачи на нескольких уровнях.

Это позволило улучшить общие метрики - менеджеры не задумывались о том, как подчинённые будут выполнять цели, а сосредотачивались на своём уровне гранулярности задач. Они могут понимать, что успех может быть в какой-то области и направлять туда агентов. То есть пространство исследуется не равномерно, а сокрытие информации уменьшает размер пространства состояний для поиска.



### Нейросети для иерархического RL

Идея выше была переиспользована в недрах DeepMind. В 2017 году вышла статья "FeUdal Networks for Hierarchical Reinforcement Learning", где предложили нейросеть с одноимённой архитектурой FeUdal Networks (FuNs).

В то время был бум обучения с подкреплением на наборе игр ATARI, так как он предоставлял разнообразные среды и задачи для тестирования и сравнения различных алгоритмов.

В простых играх типа пинг-понга удалось достичь успеха, но в более сложных играх (Месть Монтесумы) не удалось. Проблемы были когда вознаграждение поступало редко (sparse reward signal) и нужно было долгосрочно планировать. Или если нужна была память, чтобы агент обучался, какую часть опыта ему следует запомнить, только на основе редкого вознаграждения. В иерархическом (feudal) RL уровни иерархии внутри агента взаимодействуют через явные цели, которые спускаются сверху вниз и установка целей не зависит от их достижения. Уровень иерархии взаимодействует с нижележащим уровнем, указывая, что должно быть достигнуто, но не каким образом. Объявление целей высокого уровня в более низкоуровневом представлении естественным образом структурирует поведение агентов в протяженные во времени под-политики.

Ух забористо вышло выше... надо подкинуть ещё.

В данной статье высокоуровневый менеджер устанавливает цели в менее гранулярном латентном пространстве состояние-действие, которое обучается им самим. Низкоуровневый рабочий, оперирует в более детализированном пространстве и производит простые действия, в соответствии с целями, установленными менеджером. Мотивация рабочего следовать этим целям достигается через внутреннюю награду. Важно отметить, что градиенты не протекают между рабочим и менеджером. Менеджер получает свой обучающий сигнал исключительно из окружающей среды. Другими словами менеджер обучается выбирать скрытые цели, которые максимизируют внешнее вознаграждение.

Основные вклады исследования:

* сквозное дифференцирование, обобщающее принципы иерархического RL
* цели являются не абсолютными, а лишь задают направление
* LSTM для менеджера позволяет удерживать более длинный контекст

В LeakGAN была заимствована концепция из FuNs, однако, в отличие от FuNs, где функция вознаграждения является неявной (чёрным ящиком), LeakGAN использует модель дискриминатора и высокоуровневые признаки из внутреннего представления дискриминатора, передаются менеджеру. Таким образом менеджер является "шпионом", который "сливает" информацию от дискриминатора, чтобы помогать генератору.

<img src="https://raw.githubusercontent.com/sswt/dive2gai/main/.github/images/w4/leakgan_algorithm.png" alt="SeqGAN" width="75%" />



### LeakGAN-методология более детально

**TODO:** Избавиться от повторений одного и того же, некоторые мысли сейчас дублируются.

Есть последовательность $s_t = (x_1, ..., x_i, ..., x_t)$, где $x_i$ - токен из словаря $V$. Генератор $G_\theta$ соответстсвует стохастической стратегии (stochastic policy), отображает $s_t$ в распределение вероятностей по словарю, откуда сэмплируется следующий токен $s_{t+1}$.

Дискриминатор $D_\phi$ даёт скалярный управляющий сигнал $D_\phi(s_T)$ генератору для подстройки его параметров, когда вся последовательность $s_T$ уже сгенерирована. При длинных последовательностях скалярного сигнала недостаточно. Поэтому дискриминатор выдаёт генератору дополнительно вектор признаков $f_t$ в добавок к его текущему внутреннему состоянию $s_t$.

#### Фичи дискриминатора как управляющие сигналы

Обычно в RL функция вознаграждения - чёрный ящик. В LeakGAN дискриминатор используется как обучаемая функция вознаграждения. Вообще там нейросетка, которая тоже в целом чёрный ящик, но мы можем её разложить на feature extractor $\mathcal{F}(·; \phi_f)$ and a последний слой классификатора с сигмоидой и весами $\phi_l$.

$$D_\phi(s)=\sigma(\phi_l^T\mathcal{F}(s;\phi_f))=\sigma(\phi_l^Tf), \tag{1}$$ где $\phi=(\phi_f, \phi_l), \sigma(x)=1/(1+e^{-z})$, $f$ - вектор признаков, который передаётся в генератор.

Есть известное наблюдение, что нейросети для задачи классификации стремятся создать легко разделимое пространство в на выходе предпоследнего слоя. Как видно из формулы (1) вознаграждение на выходе зависит от $f$, поэтому цель получить высокое вознаграждение эквивалентно цели найти область с высоким вознаграждением в пространстве $\mathcal{F}(S; \phi_f) = \{\mathcal{F}(s; \phi_f)\}_{s \in S}$.

Стоит отметить, что сравнению со скалярным сигналом, вектор $f$ гораздо более информативен.

#### Иерархическая структура генератора

На каждом шаге $t$ во время генерации для эффективного использования $f_t$ генератором используются принципы иерархического RL. Он состоит из модуля MANAGER, который представляет собой LSTM, принимающую на вход $f_t$ на шаге $t$ и выдающую вектор целей $g_t$, который передаётся на вход модуля WORKER для управления генерацией следующего слова так, чтобы попасть в область высокого вознаграждения в пространстве $\mathcal{F}(·; \phi_f)$.

#### Процесс генерации

$M$ и $W$ начинают с нулевых скрытых состояний $h_0^W$ и $h_0^M$. На каждом шаге M из вектора $f_t$ от дискриминатора и своего скрытого состояния производит вектор цели $g_t$
$$\hat g_t, h^M_t = M(f_t, h^M_{t−1}; \theta_m), \tag{2}$$
$$g_t = \hat g_t/‖\hat g_t‖,$$
где $\theta_m$ - параметры LSTM-сети модуля $M$.

Для интеграции целей полученных $M$, линейная трансформация $\psi$ с весами $W_\psi$ выполняется над суммой последних $c$ целей, для того, чтобы получить $k$-мерный вектор эмбеддингов целей
$$w_t = \psi\Big(\sum_{i=1}^c g_{t-i}\Big)=W_\psi\Big(\sum_{i=1}^c g_{t-i}\Big)$$
Модуль $W$ получая на вход текущее слово $x_t$ и выход $O_t$, который комбинируется с вектором эмбеддингов $w_t$ с помощью матричного произведения для определения финального распределения вероятностей следующих слов через softmax:
$$O_t, h^W_t = W(x_t, h^W_{t−1}; \theta_w),$$
$$G_\theta(·|s_t) = softmax(O_t · w_t/α),$$
где $\theta_w$ - параметры LSTM-сети модуля $W$,  $O_t$ матрица $|V |×k$, $O_t · w_t$ - логиты для всех слов, $\alpha$ - температура для контроля энтропии генерации.

#### Обучение генератора

Процедура выше полностью дифференцируема. Мы можем обучать генератор от начала до конца с использованием gradient policy алгоритма, например REINFORCE. При этом нам бы хотелось, чтобы $M$ выучил какие-то значимые закономерности, поэтому, следуя алгоритму из FeUdal Networks $M$ и $W$ обучаются раздельно. $M$ обучается предсказать выгодные напаравления в дискриминативном пространстве, а $W$ получает внутреннее вознаграждение за следование этим направлениям.

Градиент $M$ определяется как

$$∇^{adv}_{θ_m} g_t = −Q_{\mathcal{F}} (s_t, g_t)∇_{θ_m} d_{cos}(f_{t+c} − f_t, g_t(θ_m)), \tag{7}$$

где $Q_{\mathcal{F}} (s_t, g_t) = Q(F(st), gt) = Q(ft, gt) = E[rt]$ - ожидаемая награда при текущей стратегии, которая может быть оценена по методу Монте-Карло; $d_{cos}$ - косинусная мера схожести между изменением $f$ за $c$-шагов ($f_{t+c} − f_t$) и вектор цели $g_t(θ_m)$ производится менеджером по формуле (2).

Интуитивно, функция потерь используется для принуждения соответствия вектора цели переходу в пространстве признаков, при этом достигая высокого вознаграждения.

В то же время, рабочий обучается максимизировать вознаграждение с использованием алгоритма REINFORCE, как это сделано в SeqGAN:

$$∇_{θ_w}\mathbb{E}_{s_{t−1}∼G}\Big[\sum_{x_t}r^I_tW(x_t|s_{t−1}; θ_w)\Big]
=\mathbb{E}_{s_{t−1}∼G,x_t∼W(xt_|s_{t−1})}\Big[r^I_t ∇_{θ_w} log W(x_t|s_{t−1}; θ_w)\Big], \tag{8}$$

что можно аппроксимировать сэмплируя состояние $s_{t-1}$ и действие $x_t$, предпринимаемое рабочим. Так как рабочий поощряется за следование направлениям, задаваемым менеджером, внутреннее вознаграждение рабочего определяется как:

$$r^I_t = \frac{1}{c} \sum_{i=1}^{c} d_{cos}(f_t - f_{t-i}, g_{t-i}). \tag{9}$$

На практике, до состязательного обучения нам нужно предобучить $G_θ$. Будем последовательными - при предобучении используется немного другая схема, где градиент менеджера выражается как

$$∇^{pre}_{θ_m}g_t = −∇_{θ_m}d_{cos}(\hat{f}_{t+c} − \hat{f}_t, g_t(θ_m)), \tag{10}$$

где $\hat{f}_t = \mathcal{F}(\hat{s}_t)$, $\hat{s}_t$ и $\hat{s}_{t+c}$ -  состояния реального текста, а значение функции состояние-действие $Q_{\mathcal{F}}(s_t, g_t)$ из выражения (7) устанавливается равным 1, так как данные примеры, используемые в предварительном обучении, представляют собой реальные предложения.

Таким образом, менеджер обучается имитировать "переходы" реальных текстовых образцов в пространстве признаков, в то время как работник обучается с использованием метода максимального правдоподобия (MLE).

В процессе обучения генератор $G_θ$ и дискриминатор $D_φ$ обучаются поочередно. В генераторе менеджер $M(· ; θ_m)$ и работник $W(· ; θ_w)$ (включая $ψ$ и softmax) обучаются поочередно, при этом фиксируется другой компонент.



График из статьи - NLL на синтетических данных, сгенерированных случайно инициализированным LSTM-оракулом.
<img src="https://raw.githubusercontent.com/sswt/dive2gai/main/.github/images/w4/leakgan_nll.png" alt="SeqGAN" width="60%" />

<a name="implementation"></a>
## Реализация

<a name="data_loaders"></a>
### Data loaders

In [ ]:
from torch.utils.data import Dataset, DataLoader

class Real_Dataset(Dataset):
    def __init__(self, filepath):
        self.data = np.load(filepath)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.from_numpy(self.data[idx]).long()

class Dis_Dataset(Dataset):
    def __init__(self, positive_filepath, negative_filepath):
        pos_data = np.load(positive_filepath, allow_pickle=True)
        neg_data = np.load(negative_filepath, allow_pickle=True)
        #print("Pos data: {}".format(len(pos_data)))
        #print("Neg data: {}".format(len(neg_data)))
        pos_label = np.array([1 for _ in pos_data])
        neg_label = np.array([0 for _ in neg_data])
        self.data = np.concatenate([pos_data, neg_data])
        self.label = np.concatenate([pos_label, neg_label])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = torch.from_numpy(self.data[idx]).long()
        label = torch.nn.init.constant_(torch.zeros(1), int(self.label[idx])).long()
        return {"data": data, "label": label}


def real_data_loader(filepath, batch_size, shuffle, num_workers, pin_memory):
    dataset = Real_Dataset(filepath)
    return DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=pin_memory)

def dis_data_loader(positive_filepath, negative_filepath, batch_size, shuffle, num_workers, pin_memory):
    dataset = Dis_Dataset(positive_filepath, negative_filepath)
    return DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=pin_memory)

<a name="generator"></a>
### Generator

In [ ]:
from scipy.stats import truncnorm
from torch.autograd import Variable
from torch.distributions import Categorical

# A truncated distribution has its domain (the x-values) restricted to a certain range of values.
# For example, you might restrict your x-values to between 0 and 100, written in math terminology as {0 > x > 100}.
# There are several types of truncated distributions:
def truncated_normal(shape, lower=-0.2, upper=0.2):
    size = 1
    for dim in shape:
        size *= dim
    w_truncated = truncnorm.rvs(lower, upper, size=size)
    w_truncated = torch.from_numpy(w_truncated).float()
    w_truncated = w_truncated.view(shape)
    return w_truncated

class Manager(nn.Module):
    def __init__(self, batch_size, hidden_dim, goal_out_size):
        super(Manager, self).__init__()
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.goal_out_size = goal_out_size
        self.recurrent_unit = nn.LSTMCell(
            self.goal_out_size, #input size
            self.hidden_dim #hidden size
        )
        self.fc = nn.Linear(
            self.hidden_dim, #in_features
            self.goal_out_size #out_features
        )
        self.goal_init = nn.Parameter(torch.zeros(self.batch_size, self.goal_out_size))
        self._init_params()

    def _init_params(self):
        for param in self.parameters():
            nn.init.normal_(param, std=0.1)
        self.goal_init.data = truncated_normal(
            self.goal_init.data.shape
        )
    def forward(self, f_t, h_m_t, c_m_t):
        """
        f_t = feature of CNN from discriminator leaked at time t, it is input into LSTM
        h_m_t = ouput of previous LSTMCell
        c_m_t = previous cell state
        """
        #print("H_M size: {}".format(h_m_t.size()))
        #print("C_M size: {}".format(c_m_t.size()))
        #print("F_t size: {}".format(f_t.size()))
        h_m_tp1, c_m_tp1 = self.recurrent_unit(f_t, (h_m_t, c_m_t))
        sub_goal = self.fc(h_m_tp1)
        sub_goal = torch.renorm(sub_goal, 2, 0, 1.0) #Returns a tensor where each sub-tensor of input along dimension dim is normalized such that the p-norm of the sub-tensor is lower than the value maxnorm
        return sub_goal, h_m_tp1, c_m_tp1

class Worker(nn.Module):
    def __init__(self, batch_size, vocab_size, embed_dim, hidden_dim,
                    goal_out_size, goal_size):
        super(Worker, self).__init__()
        self.batch_size = batch_size
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.goal_out_size = goal_out_size
        self.goal_size = goal_size

        self.emb = nn.Embedding(self.vocab_size, self.embed_dim)
        self.recurrent_unit = nn.LSTMCell(self.embed_dim, self.hidden_dim)
        self.fc = nn.Linear(self.hidden_dim, self.goal_size*self.vocab_size)
        self.goal_change = nn.Parameter(torch.zeros(self.goal_out_size, self.goal_size))
        self._init_params()

    def _init_params(self):
        for param in self.parameters():
            nn.init.normal_(param, std=0.1)
    def forward(self, x_t, h_w_t, c_w_t):
        """
            x_t = last word
            h_w_t = last output of LSTM in Worker
            c_w_t = last cell state of LSTM in Worker
        """
        x_t_emb = self.emb(x_t)
        h_w_tp1, c_w_tp1 = self.recurrent_unit(x_t_emb, (h_w_t, c_w_t))
        output_tp1 = self.fc(h_w_tp1)
        output_tp1 = output_tp1.view(self.batch_size, self.vocab_size, self.goal_size)
        return output_tp1, h_w_tp1, c_w_tp1

class Generator(nn.Module):
    def __init__(self, worker_params, manager_params, step_size):
        super(Generator, self).__init__()
        self.step_size = step_size
        self.worker = Worker(**worker_params)
        self.manager = Manager(**manager_params)

    def init_hidden(self):
        h = Variable(torch.zeros(self.worker.batch_size, self.worker.hidden_dim))
        c = Variable(torch.zeros(self.worker.batch_size, self.worker.hidden_dim))
        return h, c

    def forward(self, x_t, f_t, h_m_t, c_m_t, h_w_t, c_w_t, last_goal, real_goal, t, temperature):
        sub_goal, h_m_tp1, c_m_tp1 = self.manager(f_t, h_m_t, c_m_t)
        output, h_w_tp1, c_w_tp1 = self.worker(x_t, h_w_t, c_w_t)
        last_goal_temp = last_goal + sub_goal
        w_t = torch.matmul(
            real_goal, self.worker.goal_change
        )
        w_t = torch.renorm(w_t, 2, 0, 1.0)
        w_t = torch.unsqueeze(w_t, -1)
        logits = torch.squeeze(torch.matmul(output, w_t))
        probs = F.softmax(temperature * logits, dim=1)
        x_tp1 = Categorical(probs).sample()
        return x_tp1, h_m_tp1, c_m_tp1, h_w_tp1, c_w_tp1,\
                last_goal_temp, real_goal, sub_goal, probs, t + 1

### Discriminator

In [ ]:
class Highway(nn.Module):
    #Highway Networks = Gating Function To Highway = y = xA^T + b
    def __init__(self, in_size, out_size):
        super(Highway, self).__init__()
        self.fc1 = nn.Linear(in_size, out_size)
        self.fc2 = nn.Linear(in_size, out_size)
    def forward(self, x):
        # highway = F.sigmoid(highway)*F.relu(highway) + (1. - transform)*pred # sets C = 1 - T
        g = F.relu(self.fc1)
        t = torch.sigmoid(self.fc2)
        out = g*t + (1. - t)*x
        return out

class Discriminator(nn.Module):
    """
    A CNN for text classification
    num_filters (int): This is the output dim for each convolutional layer, which is the number
          of "filters" learned by that layer.
    """
    def __init__(self, seq_len, num_classes, vocab_size, dis_emb_dim,
                    filter_sizes, num_filters, start_token, goal_out_size, step_size, dropout_prob, l2_reg_lambda):
        super(Discriminator, self).__init__()
        self.seq_len = seq_len
        self.num_classes = num_classes
        self.vocab_size = vocab_size
        self.dis_emb_dim = dis_emb_dim
        self.filter_sizes = filter_sizes
        self.num_filters = num_filters
        self.start_token = start_token
        self.goal_out_size = goal_out_size
        self.step_size = step_size
        self.dropout_prob = dropout_prob
        self.l2_reg_lambda = l2_reg_lambda
        self.num_filters_total = sum(self.num_filters)

        #Building up layers
        self.emb = nn.Embedding(self.vocab_size + 1, self.dis_emb_dim)
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_f, (f_size, self.dis_emb_dim)) for f_size, num_f in zip(self.filter_sizes, self.num_filters)
        ])
        self.highway = nn.Linear(self.num_filters_total, self.num_filters_total)
        #in_features = out_features = sum of num_festures
        self.dropout = nn.Dropout(p = self.dropout_prob)
        #Randomly zeroes some of the elements of the input tensor with probability p using Bernouli distribution
        #Each channel will be zeroed independently onn every forward call
        self.fc = nn.Linear(self.num_filters_total, self.num_classes)

    def forward(self, x):
        """
        Argument:
            x: shape(batch_size * self.seq_len)
               type(Variable containing torch.LongTensor)
        Return:
            pred: shape(batch_size * 2)
                  For each sequence in the mini batch, output the probability
                  of it belonging to positive sample and negative sample.
            feature: shape(batch_size * self.num_filters_total)
                     Corresponding to f_t in original paper
            score: shape(batch_size, self.num_classes)

        """
        #1. Embedding Layer
        #2. Convolution + maxpool layer for each filter size
        #3. Combine all the pooled features into a prediction
        #4. Add highway
        #5. Add dropout. This is when feature should be extracted
        #6. Final unnormalized scores and predictions
        emb = self.emb(x).unsqueeze(1)
        convs = [F.relu(conv(emb)).squeeze(3) for conv in self.convs] # [batch_size * num_filter * seq_len]
        pooled_out = [F.max_pool1d(conv, conv.size(2)).squeeze(2) for conv in convs] # [batch_size * num_filter]
        pred = torch.cat(pooled_out, 1) # batch_size * sum(num_filters)
        #print("Pred size: {}".format(pred.size()))
        highway = self.highway(pred)
        #print("highway size: {}".format(highway.size()))
        highway = torch.sigmoid(highway)* F.relu(highway) + (1.0 - torch.sigmoid(highway))*pred
        features = self.dropout(highway)
        score = self.fc(features)
        pred = F.log_softmax(score, dim=1) #batch * num_classes
        return {"pred":pred, "feature":features, "score": score}

    def l2_loss(self):
        W = self.fc.weight
        b = self.fc.bias
        l2_loss = torch.sum(W*W) + torch.sum(b*b)
        l2_loss = self.l2_reg_lambda * l2_loss
        return l2_loss

### Learning

In [ ]:
def save_checkpoint(model_dict, optimizer_dict, scheduler_dict, ckpt_num, replace=False):
    file_name = "checkpoint" + str(ckpt_num) + ".pth.tar"
    torch.save({"model_dict": model_dict, "optimizer_dict": optimizer_dict, "scheduler_dict": scheduler_dict, "ckpt_num": ckpt_num}, file_name)
    if replace:
        ckpts = glob.glob("checkpoint*")
        ckpt_nums = [int(x.split('.')[0][10:]) for x in ckpts]
        oldest_ckpt = "checkpoint" + str(min(ckpt_nums)) + ".pth.tar"
        os.remove(oldest_ckpt)

def restore_checkpoint(ckpt_path):
    checkpoint = torch.load(ckpt_path)
    return checkpoint

In [ ]:
train_params = {
    "lr_dict": {"worker": 0.0015, "manager": 0.0015, "discriminator": 5e-05},
    "decay_step_size": 200,
    "decay_rate": 0.99,
    "total_epoch": 80,  # 800,
    "generated_num": 156,
    "save_num": 10,
    "replace_num": 5,
    "pre_dis_epoch_num": 20,  # 50,
    "pre_gen_epoch_num": 30,  # 80,
    "pos_filepath": "./data/train_corpus.npy",
    "neg_filepath": "./data/gen_data.npy",
    "eval_filepath": "./data/eval_data.npy",
    "model_path": "./ckpts/",
    "seed": 233,
    "checkpoint_path": None
}

target_params = {
    "vocab_size": 5000,
    "batch_size": 64,
    "embed_dim": 32,
    "hidden_dim": 32,
    "seq_len": max_len,  # 20
    "start_token": 0
}

dis_data_params = {
    "positive_filepath": "./data/train_corpus.npy",
    "negative_filepath": "./data/gen_corpus.npy",
    "batch_size": 64,
    "shuffle": True,
    "num_workers": 4,
    "pin_memory": False
}

real_data_params = {
    "filepath": "./data/train_corpus.npy",
    "batch_size": 64,
    "shuffle": True,
    "num_workers": 4,
    "pin_memory": False
}

discriminator_params = {
  "seq_len": max_len,  # 20
  "num_classes": 2,
#   "vocab_size": 5258,
  "dis_emb_dim": 64,
  "filter_sizes": [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20 ],
  "num_filters": [ 100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160, 160 ],
  "start_token": 0,
  "goal_out_size": None,
  "step_size": 5,
  "dropout_prob": 0.8,
  "l2_reg_lambda": 0.2
}

# generator_params
manager_params = {"batch_size": 64, "hidden_dim": 32, "goal_out_size": None}
worker_params = {"batch_size": 64, "vocab_size": vocab_size, "embed_dim": 32, "hidden_dim": 32, "goal_out_size": None, "goal_size": 16}
step_size = 5

In [ ]:
torch.manual_seed(train_params["seed"])

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')
use_cuda = device == torch.device('cuda')
device, use_cuda

(device(type='cuda'), True)

In [ ]:
discriminator_params["goal_out_size"] = sum(discriminator_params["num_filters"])
worker_params["goal_out_size"] = discriminator_params["goal_out_size"]
manager_params["goal_out_size"] = discriminator_params["goal_out_size"]
discriminator = Discriminator(**discriminator_params, vocab_size=vocab_size)
generator = Generator(worker_params, manager_params, step_size)
generator.to(device)
discriminator.to(device)
model_dict = {"generator": generator, "discriminator": discriminator}

In [ ]:
lr_dict = train_params["lr_dict"]
w_optimizer = torch.optim.Adam(generator.worker.parameters(), lr=lr_dict["worker"])
m_optimizer = torch.optim.Adam(generator.manager.parameters(), lr=lr_dict["manager"])
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr_dict["discriminator"])
optimizer_dict = {"worker": w_optimizer, "manager": m_optimizer, "discriminator": d_optimizer}

In [ ]:
gamma = train_params["decay_rate"]
step_size = train_params["decay_step_size"]

w_scheduler = torch.optim.lr_scheduler.StepLR(w_optimizer, step_size=step_size, gamma=gamma)
m_scheduler = torch.optim.lr_scheduler.StepLR(m_optimizer, step_size=step_size, gamma=gamma)
d_scheduler = torch.optim.lr_scheduler.StepLR(d_optimizer, step_size=step_size, gamma=gamma)
scheduler_dict = {"worker": w_scheduler, "manager": m_scheduler, "discriminator": d_scheduler}

<a name="gen_pretrain"></a>
#### Pretrain discriminator

In [ ]:
def pretrain_discriminator(model_dict, optimizer_dict, scheduler_dict,
                           dis_dataloader_params, vocab_size, positive_file,
                           negative_file, batch_size, epochs, use_cuda=False, temperature=1.0):
    discriminator = model_dict["discriminator"]

    d_optimizer = optimizer_dict["discriminator"]
    d_lr_scheduler = scheduler_dict["discriminator"]

    generate_samples(model_dict, negative_file, batch_size, use_cuda, temperature)
    dis_dataloader_params["positive_filepath"] = positive_file
    dis_dataloader_params["negative_filepath"] = negative_file
    #print(dis_dataloader_params)
    dataloader = dis_data_loader(**dis_dataloader_params) #this is where data iterator is used

    cross_entropy = nn.CrossEntropyLoss() #this one is similar to NLL (negative log likelihood)
    if use_cuda:
        cross_entropy = cross_entropy.cuda()

    for epoch in range(epochs):
        for i, sample in enumerate(dataloader):
            d_optimizer.zero_grad()
            data, label = sample["data"], sample["label"] #initialize sample variables
            data = Variable(data)
            label = Variable(label)
            if use_cuda:
                data = data.cuda()
                label = label.cuda()
            outs = discriminator(data)
            loss = cross_entropy(outs["score"], label.view(-1)) + discriminator.l2_loss()
            d_lr_scheduler.step()
            loss.backward()
            d_optimizer.step()
            if i == 63:
                print("Pre-Discriminator loss: {:.5f}".format(loss))

    model_dict["discriminator"] = discriminator
    optimizer_dict["discriminator"] = d_optimizer
    scheduler_dict["discriminator"] = d_lr_scheduler
    return model_dict, optimizer_dict, scheduler_dict

def generate_samples(model_dict, negative_file, batch_size,
                     use_cuda=False, temperature=1.0):
    neg_data = []
    for _ in range(batch_size):
        sample = get_sample(model_dict, use_cuda, temperature)
        sample = sample.cpu()
        neg_data.append(sample.data.numpy())
    neg_data = np.concatenate(neg_data, axis=0)
    np.save(negative_file, neg_data)

def get_sample(model_dict, use_cuda=False, temperature=1.0):
    return recurrent_func("gen")(model_dict, use_cuda, temperature)

In [ ]:
def recurrent_func(f_type = "pre"):
    """
    There are 3 types of recurrent function:
        1. pre = pretrain
        2. adv = adversarial train
        3. rollout = rollout for evaluate reward

    Each kind of training has its own function
    """
    if f_type == "pre":
        def func(model_dict, real_data, use_cuda, temperature = 1.0):
            """
                Get generator and discriminator
            """
            #print("After sample size: {}".format(real_data.size()))
            generator = model_dict["generator"]
            discriminator = model_dict["discriminator"]
            '''
            Initialize variables and lists for forward step.
            '''
            h_w_t, c_w_t, h_m_t, c_m_t, last_goal, real_goal, x_t = \
                init_vars(generator, discriminator, use_cuda)
            t = 0
            feature_list = []
            delta_feature_list = [] #F(St+c) - F(St) = used to calculate the gradient of manager module
            prediction_list = []
            real_goal_list = []
            batch_size = generator.worker.batch_size
            seq_len = discriminator.seq_len
            step_size = generator.step_size
            goal_out_size = generator.worker.goal_out_size
            vocab_size = discriminator.vocab_size
            """
                Forward step for pretrainning G & D
            """
            while t < seq_len + 1:
                #Extract Feature from D
                if t == 0:
                    cur_sen = Variable(nn.init.constant_(
                        torch.zeros(batch_size, seq_len), vocab_size
                    )).long()
                    #print("Batch Size: {}".format(batch_size))
                    #print("Real Data: {}".format(cur_sen.size()))
                else:
                    cur_sen = real_data[:,:t]
                    #print("Real Data: {}".format(real_data.size()))
                    #print("t: {}".format(t))
                    cur_sen = cur_sen.contiguous()
                    cur_sen = F.pad(cur_sen.view(-1, t), (0, seq_len - t), value=vocab_size)
                cur_sen.to(device, non_blocking=True)
                #print("Current sentence:{}".format(cur_sen))
                #print("Current sentence size:{}".format(cur_sen.size()))
                f_t= discriminator(cur_sen)["feature"]
                #print("F_t from discr: {}".format(f_t))
                #print("F_t from discr: {}".format(f_t.size()))
                #G forward tep
                x_t, h_m_t, c_m_t, h_w_t, c_w_t, last_goal, real_goal,\
                sub_goal, probs, t_ = generator(
                        x_t, f_t, h_m_t, c_m_t, h_w_t, c_w_t, last_goal,
                        real_goal, t, temperature
                    )
                if t % step_size == 0:
                    if t>0:
                        real_goal = last_goal
                    last_goal = Variable(torch.zeros(batch_size, goal_out_size))
                    last_goal.to(device, non_blocking=True)
                    real_goal_list.append(real_goal)
                """
                Store needed information for calculating loss function
                """
                feature_list.append(f_t)
                prediction_list.append(probs)
                if t > 0:
                    if t % step_size == 0:
                        delta_feature_list.append(f_t-feature_list[t - step_size])
                t = t_
            """
            Post process and return variables needed for calculating loss
            """
            if len(real_goal_list) == len(delta_feature_list) + 1:
                real_goal_list = real_goal_list[:-1] #exclude the last element
            prediction_list = prediction_list[:-1]
            real_goal_var = torch.stack(real_goal_list).permute(1,0,2)#stack = turn a list of PyTorch Tensors into one tensor, permute = rotating in regards to z axis
            #print("Prediction stack before stacking: {}".format(torch.stack(prediction_list).size()))
            prediction_var = torch.stack(prediction_list).permute(1,0,2)
            delta_feature_var = torch.stack(delta_feature_list).permute(1,0,2)
            """
            real_goal = g_t, prediction = generator sentence, delta_feature = F(s_(t+c))-F(s_t)
            """
            results = {"real_goal": real_goal_var,"prediction": prediction_var, "delta_feature": delta_feature_var}
            for result in results.values():
                if result.is_contiguous():
                    result = result.contiguous()
            return results
        return func

    #Adversarial Training
    elif f_type == "adv":
        def func(model_dict, use_cuda=False, temperature = 1.0):
            """
            Get G and D
            """
            generator = model_dict["generator"]
            discriminator = model_dict["discriminator"]
            h_w_t, c_w_t, h_m_t, c_m_t, last_goal, real_goal, x_t = \
                init_vars(generator, discriminator, use_cuda)
            t = 0
            feature_list = []
            delta_feature_list = [] # f_(t+c) - f_t
            delta_feature_for_worker_list = [] # f_t - f_(t-i)
            prediction_list = []
            real_goal_list = []
            all_goal_list = []
            gen_token_list = []
            batch_size = generator.worker.batch_size
            seq_len = discriminator.seq_len
            step_size = generator.step_size
            goal_out_size = generator.worker.goal_out_size
            vocab_size = discriminator.vocab_size
            """
            Perform forward step for adversarial training for discriminator and generator
            """
            while t < seq_len + 1:
                #Extract Feature from D
                if t == 0:
                    cur_sen = Variable(nn.init.constant_(
                        torch.zeros(batch_size, seq_len), vocab_size
                    )).long()
                else:
                    #print("Cur sen size before permute: {}".format(cur_sen.size()))
                    cur_sen = torch.stack(gen_token_list).permute(1,0)
                    #print("Cur sen size: {}".format(cur_sen.size()))
                    cur_sen = F.pad(cur_sen, (0, seq_len - t), value=vocab_size)
                #Why no cuda here: CHECK: ADD CUDA!!!!
                cur_sen = cur_sen.to(device, non_blocking=True)
                f_t = discriminator(cur_sen)["feature"]
                #Generator forward step
                x_t, h_m_t, c_m_t, h_w_t, c_w_t, last_goal, real_goal, sub_goal, probs, t_ = generator(x_t, f_t, h_m_t, c_m_t, h_w_t, c_w_t, last_goal, real_goal, t, temperature)
                if t % step_size == 0:
                    if t > 0:
                        real_goal = last_goal
                    last_goal = Variable(torch.zeros(batch_size, goal_out_size)).to(device, non_blocking=True)
                    real_goal_list.append(real_goal)
                #Store info for calculating loss function
                feature_list.append(f_t)
                prediction_list.append(probs)
                if t > 0:
                    if t % step_size == 0:
                        delta_feature_list.append(f_t-feature_list[t-step_size])
                        delta_feature_for_worker_list.append(f_t - feature_list[t - step_size])
                    else:
                        delta_feature_for_worker_list.append(f_t - feature_list[t - t%step_size])
                    all_goal_list.append(real_goal)
                gen_token_list.append(x_t) #next token generated by G
                t = t_
                #print("X size: {}".format(x_t.size()))
            #Post Process and return variables
            if len(real_goal_list) == len(delta_feature_list) + 1:
                real_goal_list = real_goal_list[:-1]
            prediction_list = prediction_list[:-1]
            gen_token_list = gen_token_list[:-1]
            real_goal_var = torch.stack(real_goal_list).permute(1,0,2)
            all_goal_var = torch.stack(all_goal_list).permute(1,0,2)
            prediction_var = torch.stack(prediction_list).permute(1,0,2)
            delta_feature_var = torch.stack(delta_feature_list).permute(1,0,2)
            #print(delta_feature_var)
            #print("Delta feature list size: {}".format(len(delta_feature_list)))
            #print("Gen token list size: {}".format(len(gen_token_list)))
            gen_token_var = torch.stack(gen_token_list).permute(1,0)
            #print("Gen token var after correct permute: {}".format(gen_token_var.size()))
            delta_feature_for_worker_var = torch.stack(delta_feature_for_worker_list).permute(1,0,2)
            results = {"real_goal": real_goal_var,
                        "all_goal": all_goal_var,
                        "prediction": prediction_var,
                        "delta_feature": delta_feature_var,
                        "delta_feature_for_worker": delta_feature_for_worker_var,
                        "gen_token": gen_token_var}
            for result in results.values():
                if result.is_contiguous():
                    result = result.contiguous()
            return results
        return func

    elif f_type == "rollout":
        def func(model_dict, input_x, given_num, use_cuda=False, temperature=1.0):
            #Get G and D
            generator = model_dict["generator"]
            discriminator = model_dict["discriminator"]
            #Init vairables and lists for forward step
            h_w_t, c_w_t, h_m_t, c_m_t, last_goal, real_goal, x_t = \
                init_vars(generator, discriminator, use_cuda)
            t = 0
            gen_token_list = []
            batch_size = generator.worker.batch_size
            seq_len = discriminator.seq_len
            step_size = generator.step_size
            goal_out_size = generator.worker.goal_out_size
            vocab_size = discriminator.vocab_size
            #Use input_x to perform G forward step
            while t < given_num +1:
                #Extract f_t
                if t == 0:
                    cur_sen = Variable(nn.init.constant_(torch.zeros(batch_size, seq_len), vocab_size)).long().to(device, non_blocking=True)
                else:
                    cur_sen = torch.stack(gen_token_list).permute(1,0)
                    cur_sen = F.pad(cur_sen, (0, seq_len - t), value=vocab_size)
                f_t = discriminator(cur_sen)["feature"]
                #G forward step now that you have f
                _, h_m_t, c_m_t, h_w_t, c_w_t, last_goal, real_goal,\
                sub_goal, probs, t_ = generator( x_t, f_t, h_m_t, c_m_t, h_w_t, c_w_t, last_goal, real_goal, t, temperature)
                if t % step_size == 0:
                    if t > 0:
                        real_goal = last_goal
                    last_goal = Variable(torch.zeros(batch_size, goal_out_size)).to(device, non_blocking=True)
                if t < given_num:
                    x_t = input_x[:, t].contiguous()
                    gen_token_list.append(x_t)
                t = t_
                #Perform Rollout
            while t < seq_len + 1:
                #Extract feature f_t
                if len(gen_token_list) == 0:
                    cur_sen = Variable(nn.init.constant_(torch.zeros(batch_size, seq_len), vocab_size)).long().to(device, non_blocking=True)
                else:
                    cur_sen = torch.stack(gen_token_list).permute(1,0)
                    cur_sen = F.pad(cur_sen, (0, seq_len - t + 1), value=vocab_size)
                f_t = discriminator(cur_sen)["feature"]
                #Generator forward step
                x_t, h_m_t, c_m_t, h_w_t, c_w_t, last_goal, real_goal,sub_goal, probs, t_ = generator(x_t, f_t, h_m_t, c_m_t, h_w_t, c_w_t, last_goal,
                    real_goal, t, temperature)
                if t % step_size == 0:
                    real_goal = last_goal
                last_goal = Variable(torch.zeros(
                    batch_size, goal_out_size
                )).to(device, non_blocking=True)
                gen_token_list.append(x_t)
                t = t_
            gen_token = torch.stack(gen_token_list).permute(1, 0)
            return gen_token
        return func
    elif f_type == "gen":
        def func(model_dict, use_cuda=False, temperature=1.0):
            generator = model_dict["generator"]
            discriminator = model_dict["discriminator"]
            h_w_t, c_w_t, h_m_t, c_m_t, last_goal, real_goal, x_t = \
                init_vars(generator, discriminator, use_cuda)
            t = 0
            gen_token_list = []
            batch_size = generator.worker.batch_size
            seq_len = discriminator.seq_len
            step_size = generator.step_size
            goal_out_size = generator.worker.goal_out_size
            vocab_size = discriminator.vocab_size
            #G forward
            while t < seq_len:
                #Extract f_t
                if t == 0:
                    cur_sen = Variable(nn.init.constant_(
                        torch.zeros(batch_size, seq_len), vocab_size)
                    ).long().to(device, non_blocking=True)
                else:
                    cur_sen = torch.stack(gen_token_list).permute(1, 0)
                    cur_sen = F.pad(
                        cur_sen, (0, seq_len - t), value=vocab_size
                    )
                # print('>', t, device, cur_sen.device)
                # print('>>', discriminator.emb.weight.device)
                f_t = discriminator(cur_sen)["feature"]
                #G forward step
                # for i, x in enumerate([x_t, f_t, h_m_t, c_m_t, h_w_t, c_w_t, last_goal, real_goal]):
                #     print(i, x.device, end=', ')
                x_t, h_m_t, c_m_t, h_w_t, c_w_t, last_goal, real_goal, sub_goal, probs, t_ = generator(x_t, f_t, h_m_t, c_m_t,
                        h_w_t, c_w_t, last_goal,real_goal, t, temperature)
                if t % step_size == 0:
                    if t > 0:
                        real_goal = last_goal
                        last_goal = Variable(torch.zeros(batch_size, goal_out_size)).to(device)
                    last_goal.to(device, non_blocking=True)
                gen_token_list.append(x_t)
                t = t_
            gen_token = torch.stack(gen_token_list).permute(1,0)
            return gen_token
        return func
    else:
        raise("Invalid function type")

def init_vars(generator, discriminator, use_cuda=False):
    h_w_t, c_w_t = generator.init_hidden() #worker unit of gen
    h_m_t, c_m_t = generator.init_hidden() #manager unit of gen
    last_goal = Variable(torch.zeros(generator.worker.batch_size, generator.worker.goal_out_size)) #bach_size * goal_out_size
    real_goal = generator.manager.goal_init
    x_t = Variable(nn.init.constant_(torch.Tensor(
        generator.worker.batch_size
    ), discriminator.start_token)).long()
    variables_ = [h_w_t, c_w_t, h_m_t, c_m_t, last_goal, real_goal, x_t]
    vs = []
    for var in variables_:
        var = var.to(device, non_blocking=True)
        vs.append(var)
    return vs

In [ ]:
from time import time

In [ ]:
print("#########################################################################")
print("Start Pretraining Discriminator...")
if use_cuda:
    dis_data_params["pin_memory"] = True
pos_file = dis_data_params["positive_filepath"]
neg_file = dis_data_params["negative_filepath"]
batch_size = train_params["generated_num"]
for i in range(train_params["pre_dis_epoch_num"]):
    t0 = time()
    print("Epoch: {}/{}  Pre-Discriminator".format(i, train_params["pre_dis_epoch_num"]))
    # break
    model_dict, optimizer_dict, scheduler_dict = pretrain_discriminator(
        model_dict, optimizer_dict, scheduler_dict, dis_data_params, vocab_size=vocab_size,
        positive_file=pos_file, negative_file=neg_file, batch_size=batch_size, epochs=1, use_cuda=use_cuda)
    print(f'elapsed: {time() - t0:.1f}s')
ckpt_num = 0
# save_checkpoint(model_dict, optimizer_dict, scheduler_dict, ckpt_num)

#########################################################################
Start Pretraining Discriminator...
Epoch: 0/20  Pre-Discriminator


/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Pre-Discriminator loss: 0.28354
elapsed: 147.7s
Epoch: 1/20  Pre-Discriminator
Pre-Discriminator loss: 0.12144
elapsed: 131.7s
Epoch: 2/20  Pre-Discriminator
Pre-Discriminator loss: 0.08997
elapsed: 106.0s
Epoch: 3/20  Pre-Discriminator
Pre-Discriminator loss: 0.06707
elapsed: 154.1s
Epoch: 4/20  Pre-Discriminator
Pre-Discriminator loss: 0.04591
elapsed: 160.1s
Epoch: 5/20  Pre-Discriminator
Pre-Discriminator loss: 0.03026
elapsed: 121.4s
Epoch: 6/20  Pre-Discriminator
Pre-Discriminator loss: 0.01947
elapsed: 166.7s
Epoch: 7/20  Pre-Discriminator
Pre-Discriminator loss: 0.01401
elapsed: 166.0s
Epoch: 8/20  Pre-Discriminator
Pre-Discriminator loss: 0.00858
elapsed: 131.7s
Epoch: 9/20  Pre-Discriminator
Pre-Discriminator loss: 0.00568
elapsed: 121.0s
Epoch: 10/20  Pre-Discriminator
Pre-Discriminator loss: 0.00448
elapsed: 111.1s
Epoch: 11/20  Pre-Discriminator
Pre-Discriminator loss: 0.00328
elapsed: 178.3s
Epoch: 12/20  Pre-Discriminator
Pre-Discriminator loss: 0.00306
elapsed: 132.5s
E

<a name="disc_pretrain"></a>
#### Pretrain generator

In [ ]:
from torch.nn.utils import clip_grad_norm_

def pretrain_generator(model_dict, optimizer_dict, scheduler_dict, dataloader, vocab_size, max_norm=5.0, use_cuda=False, epoch=1, tot_epochs=100):
    #get the models of generator
    generator = model_dict["generator"]
    worker = generator.worker
    manager = generator.manager
    #get the optimizers
    m_optimizer = optimizer_dict["manager"]
    w_optimizer = optimizer_dict["worker"]

    m_optimizer.zero_grad()
    w_optimizer.zero_grad()

    m_lr_scheduler = scheduler_dict["manager"]
    w_lr_scheduler = scheduler_dict["worker"]
    """
     Perform pretrain step for real data
    """

    for i, sample in enumerate(dataloader):
        #print("DataLoader: {}".format(dataloader))
        m_lr_scheduler.step()
        w_lr_scheduler.step()

        sample = Variable(sample)
        sample.to(device, non_blocking=True)

        # Calculate pretrain loss
        if (sample.size() == torch.zeros([64, 20]).size()): #sometimes smaller than 64 (16) is passed, so this if statement disables it
            #print("Sample size: {}".format(sample.size()))
            pre_rets = recurrent_func("pre")(model_dict, sample, use_cuda)
            real_goal = pre_rets["real_goal"]
            prediction = pre_rets["prediction"]
            delta_feature = pre_rets["delta_feature"]

            m_loss = loss_func("pre_manager")(real_goal, delta_feature)
            torch.autograd.grad(m_loss, manager.parameters())
            clip_grad_norm_(manager.parameters(), max_norm=max_norm)
            m_optimizer.step()
            m_optimizer.zero_grad()

            w_loss = loss_func("pre_worker")(sample, prediction, vocab_size, use_cuda)
            torch.autograd.grad(w_loss, worker.parameters())
            clip_grad_norm_(worker.parameters(), max_norm=max_norm)
            w_optimizer.step()
            w_optimizer.zero_grad()
            if i >= 0:
                print("Pre-Manager Loss: {:.5f}, Pre-Worker Loss: {:.5f}\n".format(m_loss, w_loss))
    """
    Update model_dict, optimizer_dict, and scheduler_dict
    """

    generator.woroker = worker
    generator.manager = manager
    model_dict["generator"] = generator

    optimizer_dict["manager"] = m_optimizer
    optimizer_dict["worker"] = w_optimizer

    scheduler_dict["manager"] = m_lr_scheduler
    scheduler_dict["worker"] = w_lr_scheduler

    return model_dict, optimizer_dict, scheduler_dict

def loss_func(f_type="pre_worker"):
    """
    5 kind of loss function: pre_worker, pre_manager, adv_worker, adv_manager, dis
    """
    if f_type == "pre_worker":
        def func(real_data, prediction, vocab_size, use_cuda=False):
            #print("Prediction shape before: {}".format(prediction.size()))
            prediction = torch.clamp(prediction, 1e-20, 1.0) # put min and max boundaries
            #print("One Hot: {}".format(one_hot(real_data, vocab_size, use_cuda).size()))
            #print("Real data size: {}".format(real_data.size()))
            #print("Log Prediction: {}".format(torch.log(prediction).size()))
            hot_one = one_hot(real_data, vocab_size, use_cuda)
            #print("Pred after reshape: {}".format(prediction.size()))
            #print("One Hot after reshape: {}".format(hot_one.size()))
            loss = -torch.mean(one_hot(real_data, vocab_size, use_cuda) * torch.log(prediction))
            return loss
        return func
    elif f_type == "pre_manager":
        def func(real_goal, delta_feature):
            loss = -torch.mean(1.0 - F.cosine_similarity(real_goal, delta_feature))
            return loss
        return func
    elif f_type == "adv_worker":
        def func(all_goal, delta_feature_for_worker, gen_token, prediction, vocab_size, use_cuda=False):
            intrinsic_rewards = 1.0 - F.cosine_similarity(all_goal, delta_feature_for_worker, dim=2)
            prediction = torch.clamp(prediction, 1e-20, 1.0)
            loss = -torch.mean(intrinsic_rewards * torch.sum(one_hot(gen_token, vocab_size, use_cuda)* torch.log(prediction), dim=2))
            return loss
        return func
    elif f_type == "adv_manager":
        def func(rewards, real_goal, delta_feature):
            loss = -torch.mean(rewards*(1.0 - F.cosine_similarity(delta_feature, real_goal, dim=2)))
            return loss
        return func
    elif f_type == "dis":
        def func(discriminator, input_x, score, use_cuda=False):
            """
            input_x:
                size(batch_size*seq_len)
                type(torch.LongTensor)
            score:
                size(batch_size * seq_len * vocab_size)
                type(torch.FloatTensor)
            """
            loss_func = nn.CrossEntropyLoss()
            if use_cuda:
                loss_func = loss_func.cuda()
            input_x = input_x.view(-1) #last dim
            batch_size, seq_len, vocab_size = score.size()
            score = score.view(batch_size * seq_len, -1) #reshape
            loss = loss_func(score, input_x) + discriminator.l2_loss()
            return loss
        return func
    else:
        raise("Invalid loss function type")

def one_hot(x, vocab_size, use_cuda=False):
    batch_size, seq_len = x.size()
    #print(x.device)
    out = torch.zeros(batch_size* seq_len, vocab_size, device=x.device)
    #
    # print(out.size())
    x = x.contiguous()
    x = x.view(-1, 1)
    #print("X size: {}".format(x.size()))
    #print("Out size: {}".format(out.size()))
    #print("Out size at dim 1: {}".format(out.size(1)))
    if (x.data < vocab_size).all() == 0:
        for i, d in enumerate(x.data):
            if x[i].item() > vocab_size - 1 :
                x[i] = 0
                #print(x[i])
                #print (i)
    out = out.scatter_(1, x.data, 1.0) #setting particular values of a tensor at the provided indices, one hot vector at positions where there is word
    """
        check places with 1.0 in out
        a = (out == 1.0).nonzero()
        print(a)
    """
    out = out.view(batch_size, seq_len, vocab_size)
    out = Variable(out)
    out.to(device, non_blocking=True)
    return out

In [ ]:
#Pretrain generator
print ("#########################################################################")
print ("Start Pretraining Generator...")
if use_cuda:
    real_data_params["pin_memory"] = True
r_dataloader = real_data_loader(**real_data_params)
for epoch in range(train_params["pre_gen_epoch_num"]):
    t0 = time()
    # break
    print("Epoch: {}/{}  Pre-Generator".format(epoch, train_params["pre_gen_epoch_num"]))
    model_dict, optimizer_dict, scheduler_dict = pretrain_generator(
        model_dict, optimizer_dict, scheduler_dict, r_dataloader, vocab_size=vocab_size, use_cuda=use_cuda,
        epoch=epoch, tot_epochs=range(train_params["pre_gen_epoch_num"]))
    print(f'elapsed: {time() - t0:.1f}s')
#Finish pretrain and save the checkpoint
# save_checkpoint(model_dict, optimizer_dict, scheduler_dict, ckpt_num)

#########################################################################
Start Pretraining Generator...
Epoch: 0/30  Pre-Generator
elapsed: 0.6s
Epoch: 1/30  Pre-Generator
elapsed: 0.5s
Epoch: 2/30  Pre-Generator
elapsed: 0.6s
Epoch: 3/30  Pre-Generator
elapsed: 0.6s
Epoch: 4/30  Pre-Generator
elapsed: 0.6s
Epoch: 5/30  Pre-Generator
elapsed: 0.6s
Epoch: 6/30  Pre-Generator
elapsed: 0.6s
Epoch: 7/30  Pre-Generator
elapsed: 0.6s
Epoch: 8/30  Pre-Generator
elapsed: 0.6s
Epoch: 9/30  Pre-Generator
elapsed: 0.6s
Epoch: 10/30  Pre-Generator
elapsed: 0.5s
Epoch: 11/30  Pre-Generator
elapsed: 0.5s
Epoch: 12/30  Pre-Generator
elapsed: 0.5s
Epoch: 13/30  Pre-Generator
elapsed: 0.5s
Epoch: 14/30  Pre-Generator
elapsed: 0.6s
Epoch: 15/30  Pre-Generator
elapsed: 0.4s
Epoch: 16/30  Pre-Generator
elapsed: 0.4s
Epoch: 17/30  Pre-Generator
elapsed: 0.6s
Epoch: 18/30  Pre-Generator
elapsed: 0.5s
Epoch: 19/30  Pre-Generator
elapsed: 0.5s
Epoch: 20/30  Pre-Generator
elapsed: 0.5s
Epoch: 21/30  Pre-Gener

<a name="adversarial_train"></a>
#### Adversarial train

In [ ]:
def adversarial_train(model_dict, optimizer_dict, scheduler_dict, dis_dataloader_params,
                      vocab_size, pos_file, neg_file, batch_size, gen_train_num=1,
                      dis_train_epoch=5, dis_train_num=3, max_norm=5.0,
                      rollout_num=4, use_cuda=False, temperature=1.0, epoch=1, tot_epoch=100):
    """
        Get all the models, optimizer and schedulers
    """
    generator = model_dict["generator"]
    discriminator = model_dict ["discriminator"]
    worker = generator.worker
    manager = generator.manager

    m_optimizer = optimizer_dict["manager"]
    w_optimizer = optimizer_dict["worker"]
    d_optimizer = optimizer_dict["discriminator"]

    #Why zero grad only m and w?
    m_optimizer.zero_grad()
    w_optimizer.zero_grad()

    m_lr_scheduler = scheduler_dict["manager"]
    w_lr_scheduler = scheduler_dict["worker"]
    d_lr_scheduler = scheduler_dict["discriminator"]

    #Adversarial training for generator
    for _ in range(gen_train_num):
        m_lr_scheduler.step()
        w_lr_scheduler.step()

        m_optimizer.zero_grad()
        w_optimizer.zero_grad()

        #get all the return values
        adv_rets = recurrent_func("adv")(model_dict, use_cuda)
        real_goal = adv_rets["real_goal"]
        all_goal = adv_rets["all_goal"]
        prediction = adv_rets["prediction"]
        delta_feature = adv_rets["delta_feature"]
        delta_feature_for_worker = adv_rets["delta_feature_for_worker"]
        gen_token = adv_rets["gen_token"]

        rewards = get_rewards(model_dict, gen_token, rollout_num, use_cuda)
        # NOTE: The same fix as in recurrent_func("adv") for real_goal to prevent shape mismatch
        # TODO: Why this fix is needed?
        if rewards.size()[1] == delta_feature.size()[1] + 1:
            rewards = rewards[:, :-1]
        m_loss = loss_func("adv_manager")(rewards, real_goal, delta_feature)
        w_loss = loss_func("adv_worker")(all_goal, delta_feature_for_worker, gen_token, prediction, vocab_size, use_cuda)

        torch.autograd.grad(m_loss, manager.parameters()) #based on loss improve the parameters
        torch.autograd.grad(w_loss, worker.parameters())
        clip_grad_norm_(manager.parameters(), max_norm)
        clip_grad_norm_(worker.parameters(), max_norm)
        m_optimizer.step()
        w_optimizer.step()
        print("Adv-Manager loss: {:.5f} Adv-Worker loss: {:.5f}".format(m_loss, w_loss))

    del adv_rets
    del real_goal
    del all_goal
    del prediction
    del delta_feature
    del delta_feature_for_worker
    del gen_token
    del rewards

    #Adversarial training for discriminator
    for n in range(dis_train_epoch):
        generate_samples(model_dict, neg_file, batch_size, use_cuda, temperature)
        dis_dataloader_params["positive_filepath"] = pos_file
        dis_dataloader_params["negative_filepath"] = neg_file
        dataloader = dis_data_loader(**dis_dataloader_params)

        cross_entropy = nn.CrossEntropyLoss()
        if use_cuda:
            cross_entropy = cross_entropy.cuda()
        """
        for d-steps do
            Use current G, θm,θw to generate negative examples and combine with given positive examples S
            Train discriminator Dφ for k epochs by Eq. (2)
        end for
        """
        for _ in range(dis_train_num):
            for i, sample in enumerate(dataloader):
                data, label = sample["data"], sample["label"]
                data = Variable(data).to(device, non_blocking=True)
                label = Variable(label).to(device, non_blocking=True)
                outs = discriminator(data)
                loss = cross_entropy(outs["score"], label.view(-1)) + discriminator.l2_loss()
                d_optimizer.zero_grad()
                d_lr_scheduler.step()
                loss.backward()
                d_optimizer.step()
        print("{}/{} Adv-Discriminator Loss: {:.5f}".format(n, range(dis_train_epoch),loss))
    #Save all changes
    model_dict["discriminator"] = discriminator
    generator.worker = worker
    generator.manager = manager
    model_dict["generator"] = generator

    optimizer_dict["manager"] = m_optimizer
    optimizer_dict["worker"] = w_optimizer
    optimizer_dict["discriminator"] = d_optimizer

    scheduler_dict["manager"] = m_lr_scheduler
    scheduler_dict["worker"] = w_lr_scheduler
    scheduler_dict["disciminator"] = d_lr_scheduler

    return model_dict, optimizer_dict, scheduler_dict

In [ ]:
def get_rewards(model_dict, input_x, rollout_num, use_cuda=False, temperature=1.0, delta=16.0):
    #Get G and D
    generator = model_dict["generator"]
    discriminator = model_dict["discriminator"]
    discriminator = discriminator.eval()
    #Prepare constants
    seq_len = discriminator.seq_len
    step_size = generator.step_size
    #Perform rollout and calculate reward
    rewards = []
    rollout_func = recurrent_func("rollout")
    for i in range(rollout_num):
        given_num = 0
        #print("Sequence length: {}".format(seq_len))
        #print("i stage in rollout: {}".format(i))
        while given_num < seq_len:
            sample_for_reward = rollout_func(model_dict, input_x, given_num, use_cuda, temperature)
            pred = discriminator(sample_for_reward)["pred"]
            pred = pred[:, 1].data
            if use_cuda:
                pred = pred.cpu()
            pred = pred.numpy()
            pred = pred.reshape(-1)
            if i == 0:
                rewards.append(pred)
            else:
                rewards[int(given_num/step_size -1)] += pred
            given_num += step_size
    rewards = rescale(rewards, delta) / rollout_num
    if use_cuda:
        rewards = rewards.cuda(non_blocking=True)
    discriminator = discriminator.train()
    return rewards

def rescale(rewards, delta=16.0):
    """
    Why Rescaled activation: during adversarial training of SeqGAN severe gradient vanishing occurs when D is much stronger than G, i.e. the reward is too small value to update the parameters
    and thus need to be rescaled before being fed into G.
        parameters for rewards:
            type: list
            length: seq_len / c, where c is c recent goals(steps into future)
            elements: np.array(size=batch_size)
            R(reward matrix) = expit(delta * (0.5 - rank(i)/B)), where expit, is an activation function that re-projects the equidifferent scoring based on ranking to a more effective distribution.
            In this model authors of the paper decided expit to be sigmoid function: expit = 1/(1+exp(-x))
    """
    r = np.array(rewards)
    _, batch_size = r.shape
    order = np.argsort(r)
    rank = np.argsort(order)
    rank = batch_size - rank
    rescaled_rewards = expit(delta*(0.5 - rank/batch_size))
    rescaled_rewards = np.transpose(rescaled_rewards)
    return Variable(torch.from_numpy(rescaled_rewards)).float()


In [ ]:
from scipy.special import expit

In [ ]:
ckpt_num = 1
#Adversarial train of D and G
print ("#########################################################################")
print ("Start Adversarial Training...")
save_num = train_params["save_num"] #save checkpoint after this number of repetitions
replace_num = train_params["replace_num"]

train_params["total_epoch"] = 20
for epoch in range(train_params["total_epoch"]):
    t0 = time()
    print("Epoch: {}/{}  Adv".format(epoch, train_params["total_epoch"]))
    # break
    model_dict, optimizer_dict, scheduler_dict = adversarial_train(
        model_dict, optimizer_dict, scheduler_dict, dis_data_params, vocab_size=vocab_size, pos_file=pos_file,
        neg_file=neg_file, batch_size=batch_size, use_cuda=use_cuda, epoch=epoch, tot_epoch=train_params["total_epoch"])
    print(f'elapsed: {time() - t0:.1f}s')
    if (epoch + 1) % save_num == 0:
        ckpt_num += 1
        # if ckpt_num % replace_num == 0:
        #     save_checkpoint(model_dict, optimizer_dict, scheduler_dict, ckpt_num, replace=True)
        # else:
        #     save_checkpoint(model_dict, optimizer_dict, scheduler_dict, ckpt_num)

#########################################################################
Start Adversarial Training...
Epoch: 0/20  Adv
Adv-Manager loss: -0.12296 Adv-Worker loss: 4.21129
0/range(0, 5) Adv-Discriminator Loss: 0.00092
1/range(0, 5) Adv-Discriminator Loss: 0.00073
2/range(0, 5) Adv-Discriminator Loss: 0.00072
3/range(0, 5) Adv-Discriminator Loss: 0.00048
4/range(0, 5) Adv-Discriminator Loss: 0.00041
elapsed: 1080.9s
Epoch: 1/20  Adv
Adv-Manager loss: -0.12297 Adv-Worker loss: 4.21359
0/range(0, 5) Adv-Discriminator Loss: 0.00066
1/range(0, 5) Adv-Discriminator Loss: 0.00026
2/range(0, 5) Adv-Discriminator Loss: 0.00022
3/range(0, 5) Adv-Discriminator Loss: 0.00022
4/range(0, 5) Adv-Discriminator Loss: 0.00017
elapsed: 1054.5s
Epoch: 2/20  Adv
Adv-Manager loss: -0.12294 Adv-Worker loss: 4.21366
0/range(0, 5) Adv-Discriminator Loss: 0.00013
1/range(0, 5) Adv-Discriminator Loss: 0.00012
2/range(0, 5) Adv-Discriminator Loss: 0.00009
3/range(0, 5) Adv-Discriminator Loss: 0.00009
4/range(0, 

In [ ]:
!ls -l data

total 8200
-rw-r--r-- 1 root root 5111936 Oct 12 19:21 gen_corpus.npy
-rw-r--r-- 1 root root 3276928 Oct 12 13:25 train_corpus.npy


In [ ]:
Xr = np.load('data/train_corpus.npy')
Xg = np.load('data/gen_corpus.npy')
Xr.shape, Xg.shape

((6400, 64), (9984, 64))

In [ ]:
def to_text(t):
    return ''.join(inv_vocab[i] for i in t.tolist())

In [ ]:
for i in range(3):
    print(Xg[i])
    print(to_text(Xg[i]))

[35 50 11 16 59 50 54 61 49 13  3 46  9 22 25 32 16 62 37 40  0  4 49 57
 21 62 13 48 17 30 58 28 39 48  6 55 49 13  0 20 20 59 41 25 29 24 19 39
 64 46 50 47  1 13 23 63  9 54 10 30  5 56 20 64]
Ti6Arimth8,e4GJQAuVY
.hpFu8gBOqMXg1nh8
EErZJNIDXweif 8Hv4m5O0oEw
[67 61 22 11 23 41 37 62 56 35 17 43 44  8 29 24 56 17 63 64 27 52 10 25
 62 34 14 46 12 51 37 37 38 11 17 38 25 32  3 43 11 34 15 25 53 15 14 66
 61 65 61 26 34 46 61 63 56  8 57 23 41 65  2 11]
ztG6HZVuoTBbc3NIoBvwLk5JuS9e7jVVW6BWJQ,b6S;Jl;9ytxtKSetvo3pHZx'6
[39 47  4 17 11 26 67  3 32 49 52 43 36 47 31 27 52 55 59 25 50 48 39 25
 41 22  8 45 23 38 15 29 20  9 67  5 42 62  7  9  6  0 62 40  9  4  8 15
 61  2 62 51 48 30 67 21 50 50 65  4 42 52 14 51]
Xf.B6Kz,QhkbUfPLknrJigXJZG3dHW;NE4z0au241
uY4.3;t'ujgOzFiix.ak9j


In [ ]:
for i in range(3):
    print(Xr[i])
    print(to_text(Xr[i]))

[ 1 30 55  1 21 50 55 50 61 46 53 66  1 22 46 55 46 59 42 61 46 45  1 28
 56 45 46 53 60  1 56 47  1 35 49 46 56 59 50 46 60  1 64 50 61 49  1 42
 61  1 28 56 60 61  1 18 56 62 55 61 42 43 53 66]
 On Finitely Generated Models of Theories with at Most Countably
[ 1 22 46 55 46 59 42 53 50 67 46 45  1 54 56 45 46 53 50 55 48  1 56 47
  1 46 44 56 53 56 48 50 44 42 53  1 57 56 57 62 53 42 61 50 56 55  1 45
 66 55 42 54 50 44 60  1 15  1 30 63 46 59  1 61]
 Generalized modeling of ecological population dynamics ; Over t
[ 1 22 46 55 46 59 42 61 50 55 48  1 34 62 43 60 62 59 47 42 44 46  1 20
 42 59 61 49  1 28 56 45 46 53 60  1 62 60 50 55 48  1 19 50 60 44 59 46
 61 46  1 33 46 57 59 46 60 46 55 61 42 61 50 56]
 Generating Subsurface Earth Models using Discrete Representatio


## Упражнения

Да какие упражнения, до конца дочитал - уже молодец ✊